# Create args

In [1]:
import argparse
import os 
class Args(argparse.Namespace):
    # data="/scratch/guest187/Data/val_SSA"
    data = "/scratch/guest187/Data/val_SSA/monai"
    preproc_set="val"
    data_used="SSA"
    results='/scratch/guest187/Data/val_SSA/results'
    optimiser="adam"
    criterion="dice"
    exec_mode="predict"
    seed=42
    batch_size=4
    val_batch_size=2

args=Args()

In [2]:
import sys 
sys.path.append('/home/guest187/GitRepo_Brats23/UNN_BraTS23/scripts')
import modelZoo_monai as mZoo
import data_loader as dl
from monai_functions import (
    save_checkpoint,
    define_model,
    define_dataloaders,
    model_params,
    val_params,
    train)

import numpy as np 
import matplotlib.pyplot as plt
import torch

# Checkpoint

In [3]:
pth = '/scratch/guest187/Data/train_gli_hack/results_hack_finetune_ssa/checkpoints/f0_finetune/epoch=39-dice=91.38.ckpt'
# pth = '/scratch/guest187/Data/train_all/results/test_fullRunThrough/best_metric_model_fullTest.pth'
checkpoint = torch.load(pth, map_location=torch.device('cpu'))
keys = checkpoint.keys()

for key in checkpoint.keys():
    print(key)

epoch
global_step
pytorch-lightning_version
state_dict
loops
callbacks
optimizer_states
lr_schedulers
NativeMixedPrecisionPlugin
hparams_name
hyper_parameters


In [4]:
state_dict = checkpoint["state_dict"]
print(type(state_dict))

state_dict2 = dict(state_dict)
print(type(state_dict2))


<class 'collections.OrderedDict'>
<class 'dict'>


In [5]:
import pickle
cnfgP = '/scratch/guest187/Data/train_all_nnUNET/train_all_data/results/11_3d/config.pkl'
config = pickle.load(open(cnfgP, "rb"))
print(config)

{'patch_size': [128, 128, 128], 'spacings': [1.0, 1.0, 1.0], 'n_class': 4, 'in_channels': 5}


In [6]:
patch_size, spacings = config["patch_size"], config["spacings"]
strides, kernels, sizes = [], [], patch_size[:]
while True:
    spacing_ratio = [spacing / min(spacings) for spacing in spacings]
    stride = [
        2 if ratio <= 2 and size >= 2 * 2 else 1 for (ratio, size) in zip(spacing_ratio, sizes)
    ]
    kernel = [3 if ratio <= 2 else 1 for ratio in spacing_ratio]
    if all(s == 1 for s in stride):
        break
    sizes = [i / j for i, j in zip(sizes, stride)]
    spacings = [i * j for i, j in zip(spacings, stride)]
    kernels.append(kernel)
    strides.append(stride)
    if len(strides) == 6:
        break
strides.insert(0, len(spacings) * [1])
kernels.append(len(spacings) * [3])

print(strides)
print(kernels)

[[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]]
[[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]


# Setup

In [7]:
# define inference method
from monai.inferers import sliding_window_inference
def inference(VAL_AMP, model, input):
    def _compute(input):
        '''
        roi_size – the spatial window size for inferences. 
        When its components have None or non-positives, the corresponding inputs dimension will be used. 
        if the components of the roi_size are non-positive values, the transform will use the corresponding components of img size.
        For example, roi_size=(32, -1) will be adapted to (32, 64) if the second spatial dimension size of img is 64
        '''
        return sliding_window_inference(
            inputs=input,
            roi_size=None,
            sw_batch_size=4, #sw_batch_size denotes the max number of windows per network inference iteration, not the batch size of inputs.
            predictor=model,
            overlap=0.2,
            mode='constant'
        )
    if VAL_AMP:
        with torch.cuda.amp.autocast():
            return _compute(input)
    else:
        return _compute(input)

In [8]:
validation_dir=args.data
validation_files = [os.path.join(validation_dir, file) for file in os.listdir(validation_dir)]
print(validation_files)
# checkpoint to test: finetuned hackathon model brats21
# checkpoint = '/scratch/guest187/Data/train_gli_hack/results_hack_finetune_ssa/checkpoints/f0_finetune/epoch=39-dice=91.38.ckpt'
checkpoint2 = '/scratch/guest187/Data/train_all/results/test_fullRunThrough/best_metric_model_fullTest.pth'

model, n_channels = define_model(checkpoint2)

# Load validation data to dataloader
data_transforms = dl.define_transforms(n_channels)
dataloader = dl.load_data(args, data_transforms)

val_loader = dataloader['val']

# Validation parameters
VAL_AMP, dice_metric, dice_metric_batch, post_transforms = val_params()

['/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00129-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00169-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00192-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00143-000', '/scratch/guest187/Data/val_SSA/monai/dataset.json', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00126-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00227-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00139-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00148-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00218-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00188-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00210-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00198-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00132-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00158-000', '/scratch/guest187/Data/val_SSA/monai/BraTS-SSA-00180-000']


RuntimeError: Error(s) in loading state_dict for UNet:
	Missing key(s) in state_dict: "model.1.submodule.1.submodule.1.submodule.1.submodule.0.conv.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.0.conv.bias", "model.1.submodule.1.submodule.1.submodule.1.submodule.0.adn.A.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.0.conv.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.0.conv.bias", "model.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.0.adn.A.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.conv.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.conv.bias", "model.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.adn.A.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.2.conv.weight", "model.1.submodule.1.submodule.1.submodule.1.submodule.1.submodule.2.conv.bias", "model.1.sub

In [ ]:
for step, batch in enumerate(val_loader):
    val_inputs = batch[0].to(device)------------------------------------
    print(val_inputs.shape)----------------------
    

In [ ]:
# model, n_channels = define_model(checkpoint)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
model.eval()

with torch.no_grad():
    # select one image to evaluate and visualize the model output
    for step, batch in enumerate(val_loader):
        val_input = batch[0].to(device)
        # print(val_input.shape)
        val_output = inference(VAL_AMP, model, val_input)
        val_output = post_transforms(val_output[0])
        plt.figure("image", (24, 6))
        for i in range(4):
            plt.subplot(1, 4, i + 1)
            plt.title(f"image channel {i}")
            plt.imshow(validation_dataset[6]["image"][i, :, :, 70].detach().cpu(), cmap="gray")
        plt.show()
        # visualize the 3 channels label corresponding to this image
        plt.figure("label", (18, 6))
        for i in range(3):
            plt.subplot(1, 3, i + 1)
            plt.title(f"label channel {i}")
            plt.imshow(validation_dataset[6]["label"][i, :, :, 70].detach().cpu())
        plt.show()
        # visualize the 3 channels model output corresponding to this image
        plt.figure("output", (18, 6))
        for i in range(3):
            plt.subplot(1, 3, i + 1)
            plt.title(f"output channel {i}")
            plt.imshow(val_output[i, :, :, 70].detach().cpu())
        plt.show()